In [4]:
import pandas as pd

# Load the dataset from the provided URL
url = "https://raw.githubusercontent.com/neilaxu/schulich_data_science/refs/heads/main/OMIS%206000/Assignment%202/hotels.csv"
hotels_df = pd.read_csv(url)

# Display the first few rows of the dataset
print("🔹 First 5 Rows of the Dataset:")
print(hotels_df.head())

# Display dataset information
print("\n🔹 Dataset Information:")
print(hotels_df.info())

# Display summary statistics
print("\n🔹 Summary Statistics:")
print(hotels_df.describe())

# Display unique floors to understand floor distribution
print("\n🔹 Unique Floors in the Dataset:")
print(sorted(hotels_df['Floor'].unique()))

🔹 First 5 Rows of the Dataset:
   Room_ID  Floor  Square_Feet  Cleaning_Time_Hours
0        1      3          682             1.814545
1        2      9          223             0.562727
2        3      7          506             1.334545
3        4      1          561             1.484545
4        5     14          424             1.110909

🔹 Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Room_ID              52 non-null     int64  
 1   Floor                52 non-null     int64  
 2   Square_Feet          52 non-null     int64  
 3   Cleaning_Time_Hours  52 non-null     float64
dtypes: float64(1), int64(3)
memory usage: 1.8 KB
None

🔹 Summary Statistics:
         Room_ID      Floor  Square_Feet  Cleaning_Time_Hours
count  52.000000  52.000000    52.000000            52.000000
mean   26.500000   8.307692  

# (a)	Can you explain how the costs associated with the first two regulations can be correctly tracked using only binary variables? That is, describe how the overtime constraints should work.

### Answer (a): Representing Costs via Binary Variables

We need binary variables for two cost components: overtime hours and floor assignments.

#### 1. Overtime Cost Tracking

For each attendant $i$, define:
* $o1_i$: 1 if working exactly 9 hours (1 hour overtime)
* $o2_i$: 1 if working exactly 10 hours (2 hours overtime)

Constraints:
* $o1_i + o2_i \leq 1$ (cannot work both 9 and 10 hours)
* $t_i \leq 8 + o1_i + 2o2_i$ (overtime hours tracking)

Where $t_i$ is total cleaning time for attendant $i$.

Overtime cost: $1.5w(o1_i + 2o2_i)$ where $w = \$25$

#### 2. Floor Assignment Tracking

For each attendant $i$, define:
* $f2_i$: 1 if cleaning exactly 2 floors
* $f3_i$: 1 if cleaning exactly 3 floors
* $f4_i$: 1 if cleaning exactly 4 floors

Constraints:
* $f2_i + f3_i + f4_i = y_i$ where $y_i$ is 1 if attendant $i$ works
* $\sum_k x_{i,k} = 2f2_i + 3f3_i + 4f4_i$ where $x_{i,k}$ is 1 if attendant $i$ assigned to floor $k$

Floor bonus: $75f3_i + 150f4_i$

#### Total Cost Function
For each attendant $i$:
$\text{Cost}_i = 8wy_i + 1.5w(o1_i + 2o2_i) + 75f3_i + 150f4_i$

Where:
* $8wy_i$: base pay (8 hours × $25)
* $1.5w(o1_i + 2o2_i)$: overtime pay
* $75f3_i + 150f4_i$: floor bonuses

# (b)	You will also need a binary variable, fik to capture whether attendant i has been assigned to floor k. This helps to determine how many floors attendant i has been assigned to. Write down the constraint such that if attendant i is assigned to at least one room on floor k, then fik = 1.

### Answer (b): Defining Binary Variable fik

To track which attendants are assigned to which floors, we define:
* fik is a binary variable that equals 1 if attendant i is assigned at least one room on floor k, and 0 otherwise.

The constraint ensuring proper assignment tracking is:

$\sum_{j \in R_k} x_{ij} \leq M \cdot f_{ik}$   for all i,k

Where:
* xij = 1 if attendant i is assigned to room j
* Rk = set of rooms on floor k
* M = number of rooms on floor k

This constraint ensures fik = 1 whenever attendant i is assigned any room on floor k.

# (c)	What type of constraint is the third regulation and why?

### Answer (c): Classification of the Third Regulation

The third regulation ("Each attendant can clean 3500 square feet of rooms per day. If they exceed this value, they double their regular hourly wage") is a **threshold constraint** because:

1. It establishes a threshold value (3500 square feet)
2. It doesn't prohibit exceeding the threshold
3. It imposes a cost penalty (doubled wage) when the threshold is exceeded

This differs from a capacity constraint as it doesn't set a hard limit but rather defines a point where the cost structure changes.

# (d)	Considering the cost of violating the regulations, in what order do you think penalties would be incurred if they become necessary? How would this ordering change if attendants received double their regular hourly wage (instead of time and a half) for overtime hours worked

### Answer (d): Penalty Order Analysis

#### Current Penalty Costs (with 1.5× overtime):
1. Overtime: $37.50/hour (max $75 for 2 hours)
2. Floor bonus: $75 per floor above 2 (max $150 for 4 floors)
3. Square footage penalty: doubles hourly wage ($200 to $400 for 8-hour day)

Therefore, penalties would be incurred in this order:
1. Overtime (least costly)
2. Extra floors
3. Square footage excess (most costly)

#### With 2× Overtime Pay:
- Overtime cost increases to $50/hour (max $100 for 2 hours)
- New penalty order:
1. Extra floors ($75 per floor)
2. Overtime
3. Square footage excess

The order changes because overtime becomes more expensive than the floor bonus when doubled.

# (e)	Use Gurobi to solve the binary program. What is the optimal cost, and how many total overtime hours and floor violations (in excess of two) occur across all attendants?

In [5]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

# Load dataset
url = "https://raw.githubusercontent.com/neilaxu/schulich_data_science/refs/heads/main/OMIS%206000/Assignment%202/hotels.csv"
hotels_df = pd.read_csv(url)

# Problem Parameters
num_attendants = 8
hourly_wage = 25
base_hours = 8
floor_bonus = 75
sqft_limit = 3500
overtime_multiplier = 1.5

# Extract floors and rooms
rooms = hotels_df["Room_ID"].tolist()
floors = hotels_df["Floor"].unique().tolist()

# Dictionary Mapping
room_sqft = dict(zip(rooms, hotels_df["Square_Feet"]))
room_time = dict(zip(rooms, hotels_df["Cleaning_Time_Hours"]))
room_floor = dict(zip(rooms, hotels_df["Floor"]))

# Create Gurobi Model
model = Model("Hotel_Scheduling")

# Decision Variables
x = model.addVars(num_attendants, rooms, vtype=GRB.BINARY, name="assign")  # Room assignment
# Improved overtime tracking with binary variables
o1 = model.addVars(num_attendants, vtype=GRB.BINARY, name="overtime_1")  # First overtime hour
o2 = model.addVars(num_attendants, vtype=GRB.BINARY, name="overtime_2")  # Second overtime hour
f = model.addVars(num_attendants, floors, vtype=GRB.BINARY, name="floor")  # Floor assignment
# Improved floor violation tracking
f3 = model.addVars(num_attendants, vtype=GRB.BINARY, name="floor_3")  # Third floor indicator
f4 = model.addVars(num_attendants, vtype=GRB.BINARY, name="floor_4")  # Fourth floor indicator
sq_penalty = model.addVars(num_attendants, vtype=GRB.BINARY, name="sq_penalty")  # Square footage penalty

# Objective Function: Minimize total cost
model.setObjective(
    quicksum(base_hours * hourly_wage for i in range(num_attendants)) +  # Base pay
    quicksum(hourly_wage * overtime_multiplier * (o1[i] + 2*o2[i]) for i in range(num_attendants)) +  # Overtime pay
    quicksum(floor_bonus * (f3[i] + 2*f4[i]) for i in range(num_attendants)) +  # Floor bonus
    quicksum(sq_penalty[i] * base_hours * hourly_wage for i in range(num_attendants))  # Square footage penalty
, GRB.MINIMIZE)

# Constraint: Every room must be assigned exactly once
for j in rooms:
    model.addConstr(quicksum(x[i, j] for i in range(num_attendants)) == 1, name=f"RoomAssign_{j}")

# Constraint: Work hour limit with binary overtime
for i in range(num_attendants):
    total_hours = quicksum(room_time[j] * x[i, j] for j in rooms)
    model.addConstr(total_hours <= base_hours + o1[i] + 2*o2[i], name=f"WorkLimit_{i}")
    # Ensure only one overtime state
    model.addConstr(o1[i] + o2[i] <= 1, name=f"OvertimeState_{i}")

# Constraint: Square footage limit
for i in range(num_attendants):
    total_sqft = quicksum(room_sqft[j] * x[i, j] for j in rooms)
    model.addConstr(total_sqft <= sqft_limit + (sq_penalty[i] * sqft_limit), name=f"SqFtLimit_{i}")

# Constraint: Floor assignment tracking
for i in range(num_attendants):
    for k in floors:
        model.addConstr(quicksum(x[i, j] for j in rooms if room_floor[j] == k) <= len(rooms) * f[i, k], 
                       name=f"FloorAssign_{i}_{k}")

# Constraint: Floor counting and violation tracking
for i in range(num_attendants):
    total_floors = quicksum(f[i, k] for k in floors)
    # Must clean between 2 and 4 floors
    model.addConstr(total_floors >= 2, name=f"MinFloors_{i}")
    model.addConstr(total_floors <= 4, name=f"MaxFloors_{i}")
    # Link floor count to floor violation indicators
    model.addConstr(total_floors == 2 + f3[i] + 2*f4[i], name=f"FloorCount_{i}")
    # Ensure only one violation state
    model.addConstr(f3[i] + f4[i] <= 1, name=f"FloorViolationState_{i}")

# Solve Model
model.optimize()

# Print Results
if model.status == GRB.OPTIMAL:
    print("\n✅ Optimal solution found!")
    print(f"🔹 Optimal Cost: ${model.objVal:.2f}")
    
    # Compute total overtime hours
    total_overtime_hours = sum(o1[i].x + 2*o2[i].x for i in range(num_attendants))
    
    # Compute total floor violations (floors beyond 2)
    total_floor_violations = sum(f3[i].x + f4[i].x for i in range(num_attendants))
    
    # Compute square footage violations
    total_sqft_violations = sum(sq_penalty[i].x for i in range(num_attendants))

    print(f"🔹 Total Overtime Hours: {int(total_overtime_hours)}")
    print(f"🔹 Total Floor Violations: {int(total_floor_violations)}")
    print(f"🔹 Total Square Footage Violations: {int(total_sqft_violations)}")

    # Display Room Assignments
    print("\n🔹 Room Assignments per Attendant:")
    for i in range(num_attendants):
        assigned_rooms = [j for j in rooms if x[i, j].x > 0.5]
        floors_assigned = len(set(room_floor[j] for j in assigned_rooms))
        overtime = "1 hour" if o1[i].x > 0.5 else "2 hours" if o2[i].x > 0.5 else "none"
        print(f"Attendant {i + 1}: Rooms {assigned_rooms} ({floors_assigned} floors, overtime: {overtime})")
else:
    print("\n❌ No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 220 rows, 568 columns and 2184 nonzeros
Model fingerprint: 0x3da5ae45
Variable types: 0 continuous, 568 integer (568 binary)
Coefficient statistics:
  Matrix range     [6e-01, 4e+03]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 2575.0000000
Presolve removed 8 rows and 0 columns
Presolve time: 0.01s
Presolved: 212 rows, 568 columns, 2072 nonzeros
Variable types: 0 continuous, 568 integer (568 binary)

Root relaxation: objective 1.660511e+03, 147 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

### Answer (e): Optimal Solution Results

The optimal scheduling solution for the hotel yields a total staffing cost of **$1,750.00**. The solution requires **4 total overtime hours**, distributed as one overtime hour each across four different attendants. The solution shows **0 floor violations** (no attendants assigned more than 2 floors) and **0 square footage violations** (no attendants exceeding 3500 sq ft).

# (f)	You will probably notice that Gurobi does not immediately provide the optimal solution when solving the model. Instead, it goes through numerous branch-and-bound iterations. While there are ways to enhance performance, one approach is to solve an approximation of the problem by relaxing the solution. Using Gurobi’s model.relax() procedure, what happens when you do this?

In [7]:
# Store original solution metrics
original_obj = model.ObjVal
original_time = model.Runtime
original_nodes = model.NodeCount
original_gap = model.MIPGap

print("\n=== Original Solution Metrics (from Question e) ===")
print(f"Optimal Cost: ${original_obj:.2f}")
print(f"Solution Time: {original_time:.2f} seconds")
print(f"Nodes Explored: {original_nodes}")
print(f"Optimality Gap: {original_gap*100:.2f}%")

# Create and solve relaxed model
print("\n=== Solving Relaxed Model ===")
relaxed_model = model.relax()
relaxed_model.optimize()

# Compare solutions
print("\nComparison of Original vs Relaxed Solutions:")
print(f"{'Metric':<20} {'Original':<15} {'Relaxed':<15}")
print("-" * 50)
print(f"{'Objective Value':<20} ${original_obj:.2f}{' ':>5} ${relaxed_model.ObjVal:.2f}")
print(f"{'Solve Time (sec)':<20} {original_time:.2f}{' ':>5} {relaxed_model.Runtime:.2f}")
print(f"{'Node Count':<20} {original_nodes:<15} {relaxed_model.NodeCount}")

# Additional analysis of relaxed solution characteristics
if relaxed_model.status == GRB.OPTIMAL:
    print("\nRelaxed Solution Characteristics:")
    print("- Variables are now continuous between 0 and 1")
    print("- Provides a lower bound on the optimal integer solution")
    print(f"- Gap between relaxed and integer solutions: ${original_obj - relaxed_model.ObjVal:.2f}")
    print(f"- Percentage improvement in solve time: {((original_time - relaxed_model.Runtime)/original_time)*100:.1f}%")


=== Original Solution Metrics (from Question e) ===
Optimal Cost: $1750.00
Solution Time: 3.98 seconds
Nodes Explored: 3334.0
Optimality Gap: 0.00%

=== Solving Relaxed Model ===
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 220 rows, 568 columns and 2184 nonzeros
Model fingerprint: 0x2eb7b490
Coefficient statistics:
  Matrix range     [6e-01, 4e+03]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 0 columns
Presolve time: 0.01s
Presolved: 212 rows, 576 columns, 2080 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6000000e+03   1.008523e-01   0.000000e+00      0s
       1    1.6605114e+03   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP s

### Answer (f): Analysis of LP Relaxation vs. Binary Program

The LP relaxation results in a **lower optimal cost** ($1,660.51 vs $1,750.00) because it allows fractional room assignments, which distribute workloads more efficiently. The **reduction in total overtime hours** from 4 to 3.24 suggests that forcing integer constraints in the binary program leads to inefficiencies and increased costs. However, since floor violations remain at **zero** in both cases, this constraint is strictly maintained regardless of relaxation.

This finding highlights that while the relaxed solution serves as a **useful lower bound**, it is not directly implementable due to its fractional assignments. The need for integer constraints in real-world scheduling problems leads to **higher costs but ensures feasible workforce allocation**, as demonstrated by the significant computational improvement (solving time reduced from 3.98 to 0.04 seconds with no branch-and-bound nodes needed).

# (g)	Instead of using model.relax(), manually create a linear relaxation by converting the decision variables from binary to continuous with the appropriate bounds. What is the optimal cost? Compare this to the optimal solution of the binary program. What do your findings imply about using the solution of relaxed model as an approximation to the binary program?

In [8]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

# Load dataset
url = "https://raw.githubusercontent.com/neilaxu/schulich_data_science/refs/heads/main/OMIS%206000/Assignment%202/hotels.csv"
hotels_df = pd.read_csv(url)

# Problem Parameters
num_attendants = 8
hourly_wage = 25
base_hours = 8
max_overtime_hours = 2
floor_bonus = 75
sqft_limit = 3500
overtime_multiplier = 1.5

# Extract floors and rooms
rooms = hotels_df["Room_ID"].tolist()
floors = hotels_df["Floor"].unique().tolist()

# Dictionary Mapping
room_sqft = dict(zip(rooms, hotels_df["Square_Feet"]))
room_time = dict(zip(rooms, hotels_df["Cleaning_Time_Hours"]))
room_floor = dict(zip(rooms, hotels_df["Floor"]))

# Create Gurobi Model
model_relaxed = Model("Hotel_Scheduling_Linear_Relaxation")

# **Decision Variables (Continuous Relaxation)**
x = model_relaxed.addVars(num_attendants, rooms, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="assign")  # Room assignment
o = model_relaxed.addVars(num_attendants, vtype=GRB.CONTINUOUS, lb=0, ub=max_overtime_hours, name="overtime")  # Overtime (continuous)
f = model_relaxed.addVars(num_attendants, floors, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="floor")  # Floor assignment (continuous)
v = model_relaxed.addVars(num_attendants, vtype=GRB.CONTINUOUS, lb=0, name="floor_violation")  # Floor violation (continuous)
sq_penalty = model_relaxed.addVars(num_attendants, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="sq_penalty")  # Square footage penalty

# **Objective Function: Minimize total cost**
model_relaxed.setObjective(
    quicksum(base_hours * hourly_wage for i in range(num_attendants)) +  # Base pay
    quicksum(hourly_wage * overtime_multiplier * o[i] for i in range(num_attendants)) +  # Overtime pay
    quicksum(floor_bonus * v[i] for i in range(num_attendants)) +  # Floor penalty
    quicksum(sq_penalty[i] * base_hours * hourly_wage for i in range(num_attendants))  # Square footage penalty
, GRB.MINIMIZE)

# **Constraints (Adjusted for Continuous Relaxation)**

# **Constraint: Every room must still be fully assigned**
for j in rooms:
    model_relaxed.addConstr(quicksum(x[i, j] for i in range(num_attendants)) == 1)

# **Constraint: Work hour limit (now allowing fractions)**
for i in range(num_attendants):
    total_hours = quicksum(room_time[j] * x[i, j] for j in rooms)
    model_relaxed.addConstr(total_hours <= base_hours + o[i])  # Overtime is now continuous

# **Constraint: Square footage limit (now allowing fractions)**
for i in range(num_attendants):
    total_sqft = quicksum(room_sqft[j] * x[i, j] for j in rooms)
    model_relaxed.addConstr(total_sqft <= sqft_limit + (sq_penalty[i] * sqft_limit))  # Allow continuous sqft penalty

# **Constraint: Floor assignment tracking (continuous)**
for i in range(num_attendants):
    for k in floors:
        model_relaxed.addConstr(quicksum(x[i, j] for j in rooms if room_floor[j] == k) <= len(rooms) * f[i, k])

# **Constraint: Track floor violations (continuous)**
for i in range(num_attendants):
    model_relaxed.addConstr(v[i] >= quicksum(f[i, k] for k in floors) - 2)

# **Constraint: Allow attendants to be assigned between 2 and 4 floors (continuous)**
for i in range(num_attendants):
    model_relaxed.addConstr(quicksum(f[i, k] for k in floors) >= 2)
    model_relaxed.addConstr(quicksum(f[i, k] for k in floors) <= 4)

# **Solve Model**
model_relaxed.optimize()

# **Print Relaxed Solution**
if model_relaxed.status == GRB.OPTIMAL:
    print("\n✅ Optimal solution found for Relaxed Model!")
    print(f"🔹 Relaxed Optimal Cost: ${model_relaxed.objVal:.2f}")
    
    # Compute total overtime hours (now continuous)
    relaxed_total_overtime_hours = sum(o[i].x for i in range(num_attendants))

    # Compute total floor violations (now continuous)
    relaxed_total_floor_violations = sum(v[i].x for i in range(num_attendants))

    # Compute square footage violations (now continuous)
    relaxed_total_sqft_violations = sum(sq_penalty[i].x for i in range(num_attendants))

    print(f"🔹 Relaxed Total Overtime Hours: {relaxed_total_overtime_hours:.2f}")
    print(f"🔹 Relaxed Total Floor Violations: {relaxed_total_floor_violations:.2f}")
    print(f"🔹 Relaxed Total Square Footage Violations: {relaxed_total_sqft_violations:.2f}")

else:
    print("\n❌ No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 204 rows, 552 columns and 2136 nonzeros
Model fingerprint: 0xb72cb7aa
Coefficient statistics:
  Matrix range     [6e-01, 4e+03]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 0 columns
Presolve time: 0.01s
Presolved: 196 rows, 560 columns, 2032 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6605114e+03   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

      29 DPushes remaining with DInf 0.0000000e+00                 0s
       0 DPushes remaining with DInf 0.0000000e+00                 0s

     461 PPushes remaining with PInf 0.0000000

### Answer (g): Linear Relaxation vs. Binary Model

The manually relaxed model yields an **optimal cost of $1660.51**, lower than the binary program's $1750.00. The relaxed model allows **fractional values** for overtime hours (1.61) compared to the integer-constrained model (4). Both models result in **zero floor violations and zero square footage violations**.

This comparison suggests that the **linear relaxation provides a valid lower bound for the binary model** while solving faster. However, since real-world scheduling may require integer values for assignments, the binary program remains necessary for practical implementation.

# (h)	In the previous two questions, you explored two different methods for relaxing a MILP model. Explain why these approaches yield different results?

### Answer (h): Why Do the Two Relaxation Methods Yield Different Results?

The two relaxation methods in Questions F and G produced different results because they relax the **Mixed-Integer Linear Program (MILP)** in fundamentally different ways:

1. **Gurobi's model.relax()** is a "black-box" approach that:
  - Automatically converts all integer/binary variables to continuous [0,1]
  - Preserves the original constraint structure and relationships
  - Results in higher overtime hours (3.24) due to maintaining original problem structure
  
2. **Manual relaxation** explicitly converted each binary variable:
  - Room assignments (x): from binary to continuous [0,1]
  - Overtime hours (o): from integer to continuous [0,2]
  - Floor assignments (f): from binary to continuous [0,1]
  - Violation tracking (v): from integer to continuous ≥ 0
  
This manual control over variable bounds and types enables more precise fractional assignments - attendants can be assigned partial rooms and partial hours, allowing the optimizer to find a solution with lower overtime (1.61 hours) through finer-grained workload distribution, while still achieving the same optimal cost ($1660.51).

# (i)	Now assume that attendants receive 2× their regular hourly wage (instead of time and a half) for the number of overtime hours worked. What is the optimal cost, and how many total overtime hours and floor violations (in excess of two) occur across all attendants? Compare this result to the optimal solution of the binary program in part (e). Was your intuition in part (d) correct?

In [5]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

# Load dataset
url = "https://raw.githubusercontent.com/neilaxu/schulich_data_science/refs/heads/main/OMIS%206000/Assignment%202/hotels.csv"
hotels_df = pd.read_csv(url)

# Problem Parameters
num_attendants = 8
hourly_wage = 25
base_hours = 8
max_overtime_hours = 2
floor_bonus = 75
sqft_limit = 3500
new_overtime_multiplier = 2.0  # Overtime now paid at 2× regular wage

# Extract floors and rooms
rooms = hotels_df["Room_ID"].tolist()
floors = hotels_df["Floor"].unique().tolist()

# Dictionary Mapping
room_sqft = dict(zip(rooms, hotels_df["Square_Feet"]))
room_time = dict(zip(rooms, hotels_df["Cleaning_Time_Hours"]))
room_floor = dict(zip(rooms, hotels_df["Floor"]))

# Create Gurobi Model
model = Model("Hotel_Scheduling_New_OT")

# Decision Variables
x = model.addVars(num_attendants, rooms, vtype=GRB.BINARY, name="assign")  # Room assignment
o = model.addVars(num_attendants, vtype=GRB.INTEGER, lb=0, ub=max_overtime_hours, name="overtime")  # Overtime (0,1,2 hours)
f = model.addVars(num_attendants, floors, vtype=GRB.BINARY, name="floor")  # Floor assignment
v = model.addVars(num_attendants, vtype=GRB.INTEGER, lb=0, name="floor_violation")  # Floor violation (integer)
sq_penalty = model.addVars(num_attendants, vtype=GRB.BINARY, name="sq_penalty")  # Square footage penalty indicator

# Objective Function: Minimize total cost
model.setObjective(
    quicksum(base_hours * hourly_wage for i in range(num_attendants)) +  # Base pay
    quicksum(hourly_wage * new_overtime_multiplier * o[i] for i in range(num_attendants)) +  # Overtime pay (2× multiplier)
    quicksum(floor_bonus * v[i] for i in range(num_attendants)) +  # Floor penalty
    quicksum(sq_penalty[i] * base_hours * hourly_wage for i in range(num_attendants))  # Square footage penalty
, GRB.MINIMIZE)

# Constraint: Every room must be assigned exactly once
for j in rooms:
    model.addConstr(quicksum(x[i, j] for i in range(num_attendants)) == 1, name=f"RoomAssign_{j}")

# Constraint: Work hour limit (max 10 hours with overtime in integer values)
for i in range(num_attendants):
    total_hours = quicksum(room_time[j] * x[i, j] for j in rooms)
    model.addConstr(total_hours <= base_hours + o[i], name=f"WorkLimit_{i}")  # Overtime in integer increments (0,1,2)

# Constraint: Square footage limit (double wage if exceeded)
for i in range(num_attendants):
    total_sqft = quicksum(room_sqft[j] * x[i, j] for j in rooms)
    model.addConstr(total_sqft <= sqft_limit + (sq_penalty[i] * sqft_limit), name=f"SqFtLimit_{i}")

# Constraint: Floor assignment binary condition
for i in range(num_attendants):
    for k in floors:
        model.addConstr(quicksum(x[i, j] for j in rooms if room_floor[j] == k) <= len(rooms) * f[i, k], name=f"FloorAssign_{i}_{k}")

# Constraint: Track floor violations (extra floors beyond 2)
for i in range(num_attendants):
    model.addConstr(v[i] >= quicksum(f[i, k] for k in floors) - 2, name=f"FloorViolation_{i}")

# Constraint: Each attendant must clean at least 2 and at most 4 floors
for i in range(num_attendants):
    model.addConstr(quicksum(f[i, k] for k in floors) >= 2, name=f"MinFloors_{i}")
    model.addConstr(quicksum(f[i, k] for k in floors) <= 4, name=f"MaxFloors_{i}")

# Solve Model
model.optimize()

# Print Results
if model.status == GRB.OPTIMAL:
    print("\n✅ Optimal solution found with Updated Overtime Multiplier!")
    print(f"🔹 Optimal Cost: ${model.objVal:.2f}")
    
    # Compute total overtime hours (must be an integer)
    total_overtime_hours = sum(int(o[i].x) for i in range(num_attendants))  # Ensuring integer output
    
    # Compute total floor violations (must be an integer)
    total_floor_violations = sum(int(v[i].x) for i in range(num_attendants))  # Ensuring integer output

    # Compute square footage violations
    total_sqft_violations = sum(int(sq_penalty[i].x) for i in range(num_attendants))  # Ensuring integer output

    print(f"🔹 Total Overtime Hours: {total_overtime_hours}")
    print(f"🔹 Total Floor Violations: {total_floor_violations}")
    print(f"🔹 Total Square Footage Violations: {total_sqft_violations}")

    # Display Room Assignments
    print("\n🔹 Room Assignments per Attendant:")
    for i in range(num_attendants):
        assigned_rooms = [j for j in rooms if x[i, j].x > 0.5]
        print(f"Attendant {i + 1}: Rooms {assigned_rooms}")
else:
    print("\n❌ No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 204 rows, 552 columns and 2136 nonzeros
Model fingerprint: 0xe3ddbcc9
Variable types: 0 continuous, 552 integer (536 binary)
Coefficient statistics:
  Matrix range     [6e-01, 4e+03]
  Objective range  [5e+01, 2e+02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 3550.0000000
Presolve removed 16 rows and 0 columns
Presolve time: 0.00s
Presolved: 188 rows, 552 columns, 1912 nonzeros
Variable types: 0 continuous, 552 integer (536 binary)

Root relaxation: objective 1.680682e+03, 160 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

  

In [10]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

# Load dataset
url = "https://raw.githubusercontent.com/neilaxu/schulich_data_science/refs/heads/main/OMIS%206000/Assignment%202/hotels.csv"
hotels_df = pd.read_csv(url)

# Problem Parameters
num_attendants = 8
hourly_wage = 25
base_hours = 8
max_overtime_hours = 2
floor_bonus = 75
sqft_limit = 3500
overtime_multiplier = 2.0  # Changed to 2× for overtime

# Extract floors and rooms
rooms = hotels_df["Room_ID"].tolist()
floors = hotels_df["Floor"].unique().tolist()

# Dictionary Mapping
room_sqft = dict(zip(rooms, hotels_df["Square_Feet"]))
room_time = dict(zip(rooms, hotels_df["Cleaning_Time_Hours"]))
room_floor = dict(zip(rooms, hotels_df["Floor"]))

# Create Gurobi Model
model = Model("Hotel_Scheduling_Double_OT")

# Decision Variables (Using your original structure)
x = model.addVars(num_attendants, rooms, vtype=GRB.BINARY, name="assign")
o = model.addVars(num_attendants, vtype=GRB.INTEGER, lb=0, ub=max_overtime_hours, name="overtime")
f = model.addVars(num_attendants, floors, vtype=GRB.BINARY, name="floor")
v = model.addVars(num_attendants, vtype=GRB.INTEGER, lb=0, name="floor_violation")
sq_penalty = model.addVars(num_attendants, vtype=GRB.BINARY, name="sq_penalty")

# Objective Function
model.setObjective(
    quicksum(base_hours * hourly_wage for i in range(num_attendants)) +
    quicksum(hourly_wage * overtime_multiplier * o[i] for i in range(num_attendants)) +
    quicksum(floor_bonus * v[i] for i in range(num_attendants)) +
    quicksum(sq_penalty[i] * base_hours * hourly_wage for i in range(num_attendants))
, GRB.MINIMIZE)

# Constraints (Using your original structure)
for j in rooms:
    model.addConstr(quicksum(x[i, j] for i in range(num_attendants)) == 1)

for i in range(num_attendants):
    total_hours = quicksum(room_time[j] * x[i, j] for j in rooms)
    model.addConstr(total_hours <= base_hours + o[i])
    
    total_sqft = quicksum(room_sqft[j] * x[i, j] for j in rooms)
    model.addConstr(total_sqft <= sqft_limit + (sq_penalty[i] * sqft_limit))
    
    for k in floors:
        model.addConstr(quicksum(x[i, j] for j in rooms if room_floor[j] == k) <= len(rooms) * f[i, k])
    
    model.addConstr(v[i] >= quicksum(f[i, k] for k in floors) - 2)
    
    model.addConstr(quicksum(f[i, k] for k in floors) >= 2)
    model.addConstr(quicksum(f[i, k] for k in floors) <= 4)

# Solve
model.optimize()

# Enhanced Output
if model.status == GRB.OPTIMAL:
    print("\n=== Results with Double Overtime Rate (2×) ===")
    print(f"Optimal Cost: ${model.objVal:.2f}")
    
    # Calculate detailed metrics
    total_overtime = sum(o[i].x for i in range(num_attendants))
    floor_violations = sum(v[i].x for i in range(num_attendants))
    sqft_violations = sum(sq_penalty[i].x for i in range(num_attendants))
    
    print("\nDetailed Metrics:")
    print(f"Total Overtime Hours: {total_overtime:.1f}")
    print(f"Floor Violations: {floor_violations:.1f}")
    print(f"Square Footage Violations: {sqft_violations:.1f}")
    
    # Per attendant detailed breakdown
    print("\nPer Attendant Breakdown:")
    for i in range(num_attendants):
        assigned_rooms = [j for j in rooms if x[i, j].x > 0.5]
        assigned_floors = len(set(room_floor[j] for j in assigned_rooms))
        overtime = f"{int(o[i].x)} hours" if o[i].x > 0 else "none"
        total_sqft = sum(room_sqft[j] for j in assigned_rooms)
        
        print(f"Attendant {i+1}:")
        print(f"  Rooms: {sorted(assigned_rooms)} ({len(assigned_rooms)} rooms)")
        print(f"  Floors: {assigned_floors} floors")
        print(f"  Overtime: {overtime}")
        print(f"  Total Square Feet: {total_sqft:.1f}")
        
    # Compare with original solution from part (e)
    print("\nComparison with Part (e) Solution:")
    print("Metric          Part (e)    Part (i)")
    print("-" * 40)
    print(f"Cost           $1750.00    ${model.objVal:.2f}")
    print(f"Overtime Hours     4.0        {total_overtime:.1f}")
    print(f"Floor Violations   0.0        {floor_violations:.1f}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 204 rows, 552 columns and 2136 nonzeros
Model fingerprint: 0x1999bb59
Variable types: 0 continuous, 552 integer (536 binary)
Coefficient statistics:
  Matrix range     [6e-01, 4e+03]
  Objective range  [5e+01, 2e+02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 3400.0000000
Presolve removed 16 rows and 0 columns
Presolve time: 0.00s
Presolved: 188 rows, 552 columns, 1912 nonzeros
Variable types: 0 continuous, 552 integer (536 binary)

Root relaxation: objective 1.680682e+03, 160 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

  

### Answer to Question I: Impact of Doubling Overtime Pay

With the overtime multiplier increased to **2×** instead of **1.5×**, the **optimal cost increased** from $1750.00 (Question E) to $1775.00. However, **total overtime hours decreased from 4 to 2**, suggesting that the model reduced reliance on overtime due to its higher cost. Additionally, **one floor violation occurred**, whereas there were **zero floor violations in Question E**.

#### Comparison to Question (D)
In Question (D), we predicted that:
1. With **1.5× overtime pay**, the model would prefer using **overtime first**, followed by **floor bonuses**, and lastly **square footage penalties** due to cost efficiency.
2. With **2× overtime pay**, **floor bonuses would become preferable over overtime**, making attendants more likely to **exceed floor limits** before relying on overtime.

The results of Question (i) **confirm this prediction**:
* Total overtime hours decreased, indicating overtime became less attractive
* A floor violation occurred, showing the model preferred exceeding floor limits rather than increasing overtime
* No square footage violations occurred, confirming this remains the least preferred option

#### Conclusion
This finding reinforces that **higher overtime pay discourages overtime use**, leading the optimization model to **shift towards violating floor constraints instead**. The cost hierarchy outlined in Question D correctly predicted this behavior, demonstrating how regulatory cost structures directly influence optimal scheduling decisions.